В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [3]:
dataname = 'shoppers_ON_QnSC_200k'
dataname_initial = 'shoppers'
model_short = 'tabddpm_ON_QnSC'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_200k'

idx = list(range(10))
constants = [0, 0.001, 0.01, 0.1, 0.25, 0.5]

CONFIG.add_arg('save_cat', None)
target_cat=True

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [5]:
model_save_path_hist = []
for sigma in constants:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [6]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/shoppers_ON_QnSC_200k` folder


In [7]:
CONFIG.get_all_args()

{'dataname': 'shoppers_ON_QnSC_200k',
 'method': None,
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': None,
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 66,
 'real_path': './synthetic/shoppers_ON_QnSC_200k/initial_real.csv',
 'test_path': './synthetic/shoppers_ON_QnSC_200k/initial_test.csv',
 'info_path': './data/shoppers_ON_QnSC_200k/initial_info.json',
 'save_path': None,
 'num_clusters': 25,
 'save_cat': None}

### Sample + Eval

In [8]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    print(model_save_path, model_name, sample_save_path)

    tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers, target_cat=target_cat)

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=False)
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

./models/tabddpm_ON_QnSC/ckpt/shoppers/tabddpm_ON_QnSC_200k_const_0 tabddpm_ON_QnSC_200k_const_0 ./synthetic/shoppers_ON_QnSC_200k/tabddpm_ON_QnSC_200k_const_0.csv
No NaNs in numerical features, skipping
mlp
Sample timestep    0
Sample timestep    0
Shape torch.Size([11097, 76])
Sampling time: 96.91635203361511
Sample path changed to ./synthetic/shoppers_ON_QnSC_200k/initial_tabddpm_ON_QnSC_200k_const_0.csv


/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

TABDDPM_ON_QNSC_200K_CONST_0: SHOPPERS_ON_QNSC_200K Dataset
Original Logistic: 0.674
Synthetic Logistic: 0.692
Original Tree: 0.630
Synthetic Tree: 0.607 

TABDDPM_ON_QNSC_200K_CONST_0 Accuracy Loss: SHOPPERS_ON_QNSC_200K
Logistic: -2.592%
Tree: 3.609% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 18/18 [00:00<00:00, 155.59it/s]|
Column Shapes Score: 98.4%

(2/2) Evaluating Column Pair Trends: |██████████| 153/153 [00:01<00:00, 117.13it/s]|
Column Pair Trends Score: 97.75%

Overall Score (Average): 98.07%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_CONST_0: 1.604 ± 0.877
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_CONST_0: 2.251 ± 1.606


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


...

TABDDPM_ON_QNSC_200K_MULT_9: SHOPPERS_ON_QNSC_200K Dataset
Original Logistic: 0.674
Synthetic Logistic: 0.689
Original Tree: 0.637
Synthetic Tree: 0.564 

TABDDPM_ON_QNSC_200K_MULT_9 Accuracy Loss: SHOPPERS_ON_QNSC_200K
Logistic: -2.207%
Tree: 11.512% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 18/18 [00:00<00:00, 134.10it/s]|
Column Shapes Score: 98.11%

(2/2) Evaluating Column Pair Trends: |██████████| 153/153 [00:01<00:00, 79.43it/s]|
Column Pair Trends Score: 97.53%

Overall Score (Average): 97.82%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_200K_MULT_9: 1.888 ± 0.886
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_200K_MULT_9: 2.469 ± 1.457


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:07<00:00,  4.85it/s]


Saving scores to  eval/mle/shoppers_ON_QnSC_200k/tabddpm_ON_QnSC_200k_mult_9.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_200K_MULT_9: 0.928 ± 0.004
shoppers_ON_QnSC_200k, tabddpm_ON_QnSC_200k_mult_9: 0.9229701526291187
n_clusters: 25
DCR Score, a value closer to 0.5 is better
shoppers_ON_QnSC_200k-tabddpm_ON_QnSC_200k_mult_9, DCR Score = 0.5142831395872759
=========== All Features ===========
Data shape:  (11097, 77)
alpha precision: 0.946370, beta recall: 0.535634


In [9]:
final_metrics_table = []
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname_initial}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [11]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
             Model                              Type      Data   
0  tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_001  shoppers   
0  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_const_0  shoppers   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_1  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_3  shoppers   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_01  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_4  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_8  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_2  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_9  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_7  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_6  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_1  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_5  shoppers   
0  tabddpm_ON_QnSC   tabddpm_ON_QnSC_200k_const_0_25  shoppers   
0  tabddpm_ON_QnSC       tabddpm_ON_QnSC_200k_mult_0  shoppers   
0  tabddpm_ON_QnSC    tabddpm_ON_QnSC_200k_const_0_5  shoppers   

       base_metrics                                                  \
  Original Logistic Synthetic Logistic Original Tree Synthetic Tree   
0            0.6742             0.6870        0.6374         0.5811   
0            0.6742             0.6916        0.6297         0.6070   
0            0.6742             0.6768        0.6374         0.6027   
0            0.6742             0.6856        0.6374         0.5780   
0            0.6742             0.6827        0.6374         0.6274   
0            0.6742             0.6710        0.6374         0.5779   
0            0.6742             0.6797        0.6374         0.6041   
0            0.6742             0.6797        0.6374         0.6194   
0            0.6742             0.6890        0.6374         0.5640   
0            0.6742             0.6769        0.6374         0.6150   
0            0.6742             0.6771        0.6374         0.5587   
0            0.6742             0.6968        0.6374         0.5969   
0            0.6742             0.6695        0.6374         0.6077   
0            0.6742             0.6594        0.6374         0.6134   
0            0.6742             0.6696        0.6374         0.5591   
0            0.6742             0.6739        0.6374         0.6032   

                                                              similarity  \
  Accuracy Loss Logistic, % Accuracy Loss Tree, % Column Shapes Score, %   
0                   -1.8986                8.8304                98.4530   
0                   -2.5918                3.6088                98.3960   
0                   -0.3905                5.4418                98.4235   
0                   -1.6958                9.3167                98.4040   
0                   -1.2691                1.5696                98.5772   
0                    0.4731                9.3329                98.2763   
0                   -0.8153                5.2158                98.2007   
0                   -0.8279                2.8206                98.3885   
0                   -2.2073               11.5117                98.1121   
0                   -0.4103                3.5033                98.3955   
0                   -0.4306               12.3442                98.0295   
0                   -3.3635                6.3513                98.3534   
0                    0.6839                4.6524                98.3079   
0                    2.1907                3.7611                97.8142   
0                    0.6783               12.2806                98.2167   
0                    0.0362                5.3539                96.4275   

                                                          \
  Column Pair Trends Score, % Overall Score (Average), %   
0                     97.8556                    98.1543   
0                     97.7493                  

In [12]:
import os
if not os.path.exists(f'./eval/total/{dataname_initial}'):
    os.makedirs(f'./eval/total/{dataname_initial}')
final_metrics_table.to_csv(f'./eval/total/{dataname_initial}/{model_short}{model_name_postfix}_final_metrics_table.csv', index=False)